# Convolutional Neural Networks

### Import TensorFlow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

from tensorflow.keras import datasets, layers, models

### Download and prepare the MNIST dataset

In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [3]:
from keras.utils import to_categorical
#one-hot encode target column
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Using TensorFlow backend.


In [4]:
train_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

### Create the convolutional base

The 6 lines of code below define the convolutional base using a common pattern: a stack of [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) and [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to color channels, MNIST has one (because the images are grayscale), whereas a color image has three (R,G,B). In this example, we will configure our CNN to process inputs of shape (28, 28, 1), which is the format of MNIST images. We do this by passing the argument `input_shape` to our first layer.



In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape = (None, None,1)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(10, (2, 2), activation='relu'))
model.add(layers.GlobalMaxPooling2D())
model.add(layers.Flatten())
# model.add(layers.MaxPooling2D((2, 2)))


Let display the architecture of our model so far.

In [0]:
model.summary()

Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as we go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically,  as the width and height shrink, we can afford (computationally) to add more output channels in each Conv2D layer.

### Add Dense layers on top
To complete our model, we will feed the last output tensor from the convolutional base (of shape (3, 3, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, we will flatten (or unroll) the 3D output to 1D,  then add one or more Dense layers on top. MNIST has 10 output classes, so we use a final Dense layer with 10 outputs and a softmax activation.

In [0]:
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(10, activation='softmax'))

 Here's the complete architecture of our model.

In [0]:
# model.summary()

As you can see, our (3, 3, 64) outputs were flattened into vectors of shape (576) before going through two Dense layers.

### Compile and train the model

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(zoom_range = [1,1.5], rotation_range = 20)
datagen.fit(train_images)

In [0]:
model.compile(optimizer='adam',
              loss='logcosh',
              metrics=['accuracy'])

epochs = 50
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(train_images, train_labels, batch_size=128):
        
        model.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(train_images) / 128:

            break


In [0]:
# !git clone -b dev https://github.com/gleblo/msu_science_work/
# import msu_science_work.mnist_handling as mn

import os
import random

dir_name = "msu_science_work/размеченное"
lst = os.listdir(dir_name)

test_list = ["msu_science_work/размеченное/" + name for name in lst]
# random.shuffle(test_list)

In [0]:
test_list.remove('msu_science_work/размеченное/img_10.jpg')

#Точность на вырезанных картинках

In [29]:
import cv2
y_pred = []
y_true = []
import numpy as np
for img in test_list:
            image = cv2.imread(img)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            new_image = (1 - gray.copy()/255)[np.newaxis,:,:, np.newaxis]
            # print(new_image)
            # print(new_image.shape)
            # plt.imshow(np.squeeze(new_image), cmap='gray')
            # plt.show()    
            # aa = sess.run(fin, feed_dict={xx: new_image})
            # # print(aa)
            num = int(img.split("/")[2][0])
            aa = model.predict(new_image)
            y_pred.append(np.argmax(aa))
            y_true.append(num)
from sklearn.metrics import accuracy_score as accuracy


print(accuracy(y_pred, y_true))


0.5573770491803278


In [19]:
answers[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.27650136, 0.        , 0.08499631, 0.        ]],
      dtype=float32)

### Evaluate the model

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [0]:
print(test_acc)

As you can see, our simple CNN has achieved a test accuracy of over 99%. Not bad for a few lines of code! For another style of writing a CNN (using the Keras Subclassing API and a GradientTape) head [here](https://github.com/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/quickstart/advanced.ipynb).